In [18]:
import sys
from typing import Callable, Dict, Iterator, Tuple
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
import boto3
from pyspark.sql.types import Row, StringType
from pyspark.sql.functions import concat_ws, col, coalesce, to_date, split, year


In [3]:
def init() -> Tuple[GlueContext, Job]:
    params = []
    if '--JOB_NAME' in sys.argv:
        params.append('JOB_NAME')
    args = getResolvedOptions(sys.argv, params)

    context = GlueContext(SparkContext.getOrCreate())
    job = Job(context)

    if 'JOB_NAME' in args:
        jobname = args['JOB_NAME']
    else:
        jobname = "test"
    job.init(jobname, args)
    return (context, job)

context, job = init()

In [4]:
sources = [
    "tonberry_bonds",
    "tonberry_earnings",
    "tonberry_economic_events",
    "tonberry_eod_prices",
    "tonberry_macro_indicators",
    "tonberry_news",
    "tonberry_news_yearly"
]

In [6]:
def get_source_node(source: str) -> list:   
    print(f"fetching {source}")
    return context.create_dynamic_frame.from_catalog(
        database="publish_all",
        table_name=source,
        transformation_ctx=f"{source}_source_node",
    )

source_nodes = { source: get_source_node(source) for source in sources }

fetching source
fetching source
fetching source
fetching source
fetching source
fetching source
fetching source


In [ ]:
for k,v in source_nodes.items():
    v.printSchema()

# resolved_frame = publish_all_dynamodb_node.resolveChoice(
#     specs=[
#         ("value", "cast:double")
#     ]
# )

# df = resolved_frame.toDF().withColumn("date", to_date(col("date")))
# df = df.withColumn("year", year(col("date")))

# transform_node = DynamicFrame.fromDF(df, context, "transformed")
# transform_node.printSchema()

root
|-- open: double
|-- low: double
|-- adjusted_close: double
|-- date: string
|-- close: double
|-- volume: long
|-- high: double
|-- partition_0: string
|-- partition_1: string

root
|-- report_date: string
|-- estimate: double
|-- code: string
|-- before_after_market: string
|-- date: string
|-- currency: string
|-- percent: double
|-- actual: double
|-- difference: double
|-- partition_0: string

root
|-- comparison: string
|-- previous: double
|-- estimate: double
|-- country: string
|-- change: double
|-- change_percentage: double
|-- date: date
|-- type: string
|-- actual: double
|-- period: string
|-- partition_0: string

root
|-- open: double
|-- low: double
|-- adjusted_close: double
|-- date: date
|-- close: double
|-- volume: long
|-- high: double
|-- partition_0: string
|-- partition_1: string

root
|-- description: string
|-- country_code: string
|-- date: string
|-- value: struct
|    |-- double: double
|    |-- long: long
|-- period: string
|-- country_name: string
|

In [19]:
source_nodes['tonberry_news'] = DynamicFrame.fromDF(source_nodes['tonberry_news'].toDF().withColumn('tags', col('tags').cast(StringType())), context, "news-fix")

In [20]:
partitioned_dataframes = { k: DynamicFrame.fromDF(v.toDF().repartition(1), context, f"{k}_partitioned_df") for k,v in source_nodes.items() }

In [21]:
for source, frame in partitioned_dataframes.items():
    context.write_dynamic_frame.from_options(
        frame=frame,
        connection_type="s3",
        format="parquet",
        connection_options={
            "path": f"s3://tonberry-feature-data-staging/feature={source}",
            "compression": "gzip"
        },
        transformation_ctx="S3bucket_node3",
    )

Py4JJavaError: An error occurred while calling o313.pyWriteDynamicFrame.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:195)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:185)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:223)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:220)


In [ ]:
job.commit()
